In [3]:

import gym
import tensorflow as tf
import numpy as np

In [4]:
env = gym.make('CartPole-v0')
action_size = env.action_space
state_size = env.observation_space
print("A: ",action_size)
print("S: ", state_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
A:  Discrete(2)
S:  Box(4,)


In [5]:

from collections import deque
class Replay_Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
            
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx]

In [6]:
action_size = 2 
state_size = 4
learning_rate = 0.01

episodes = 1000
episodic_steps = 200
gamma= 0.99

#exploartion parameters
epsilon = 1.0 # probability of exploration is 1 in beginning for doing random exploration
max_epsilon = 1.0
min_epsilon = 0.001 # close to 0, for exploitation
decay_Rate = 0.005 

#Network parameters
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001 

# Memory parameters
memory_size = 10000            # memory capacity
batch_size = 20                # experience mini-batch size
pretrain_length = batch_size   # number experiences to pretrain the memory



In [7]:
class QNetwork:
    def __init__(self, learning_rate, state_size, 
                 action_size, hidden_size, 
                 name='QNetwork'):
        
        with tf.variable_scope(name):
            # state inputs to the Q-network
            self.inputs_ = tf.placeholder(tf.float32, [None, state_size], name='inputs')
            
            # One hot encode the actions to later choose the Q-value for the action
            self.actions_ = tf.placeholder(tf.int32, [None], name='actions')
            one_hot_actions = tf.one_hot(self.actions_, action_size)
            
            # Target Q values for training
            self.targetQs_ = tf.placeholder(tf.float32, [None], name='target')
            
            # ReLU hidden layers
            self.fc1 = tf.contrib.layers.fully_connected(self.inputs_, hidden_size)
            self.fc2 = tf.contrib.layers.fully_connected(self.fc1, hidden_size)

            # Linear output layer
            self.output = tf.contrib.layers.fully_connected(self.fc2, action_size, 
                                                            activation_fn=None)
            
            ### Train with loss (targetQ - Q)^2
            # output has length 2, for two actions. This next line chooses
            # one value from output (per row) according to the one-hot encoded actions.
            self.Q = tf.reduce_sum(tf.multiply(self.output, one_hot_actions), axis=1)
            
            self.loss = tf.reduce_mean(tf.square(self.targetQs_ - self.Q))
            self.opt = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [9]:
tf.reset_default_graph()
Deep_Q_Network = QNetwork(name='main', action_size = action_size, state_size = state_size,hidden_size=hidden_size, learning_rate=learning_rate)

In [10]:
env.reset()
#take random step
s,r,d,i = env.step(env.action_space.sample())
memory = Replay_Memory(max_size=memory_size)\

#storing experiences by doing few random actions

for ii in range(pretrain_length):
    env.render()
    action = env.action_space.sample()
    s_,r,d,i = env.step(action)
    if d:
        s_ = np.zeros(s.shape)
        memory.add((s,action,r,s_))
        #start new episode
        env.reset()
        s,r,d,i = env.step(env.action_space.sample())
    else:
        memory.add((s,action,r,s_))
        s = s_
        

In [16]:
saver = tf.train.Saver()
rewards = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 0
    for ep in range(1, episodes):
        total_reward = 0
        t = 0
        while t < episodic_steps:
            step += 1
            explore = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_Rate * step)
            if explore > np.random.rand():
                # make a random action
                action = env.action_space.sample()
            else:
                # exploitiation, get values from the Q-network
                feed = {Deep_Q_Network.inputs_: s.reshape((1, *s.shape))}
                Qs = sess.run(Deep_Q_Network.output, feed_dict=feed)
                action = np.argmax(Qs)
            s_, r, d, i = env.step(action)
            total_reward += r

            if d:
                s_ = np.zeros(s.shape)
                t = episodic_steps

                print('Episode: {}'.format(ep),
                      'Total reward: {}'.format(total_reward),
                      'Training loss: {:.4f}'.format(loss),
                      'Explore P: {:.4f}'.format(explore))
                rewards.append(ep, total_reward)

                memory.add((s, action, r, s_))

                # newepisode
                env.reset()
                s, r, d, i = env.step(env.action_space.sample())
            else:
                memory.add((s, action, r, s_))
                s = s_
                t += 1

            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            rewards = np.array([each[2] for each in batch])
            next_states = np.array([each[3] for each in batch])

            # train network
            target_Qs = sess.run(Deep_Q_Network.output, feed_dict={Deep_Q_Network.inputs_: next_states})
            # Set target_Qs to 0 for states where episode ends
            episode_ends = (next_states == np.zeros(states[0].shape)).all(axis=1)
            target_Qs[episode_ends] = (0, 0)

            targets = rewards + gamma * np.max(target_Qs, axis=1)
            loss, _ = sess.run([Deep_Q_Network.loss, Deep_Q_Network.opt],
                               feed_dict={Deep_Q_Network.inputs_: states,
                                          Deep_Q_Network.targetQs_: targets,
                                          Deep_Q_Network.actions_: actions})
    saver.save(sess, "checkpoints/cartpole.ckpt")

WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
Episode: 1 Total reward: 0.0 Training loss: 1.0575 Explore P: 0.9950
